In [1]:
from transformers import MaskFormerForInstanceSegmentation, MaskFormerImageProcessor
from torch.utils.data import DataLoader
from torch.utils.tensorboard import SummaryWriter
from evaluate import load

import sys
sys.path.append('..')

from training.dataset import SemanticSegmentationDataset
from training.trainer import SegmenterModeltrainer

/home/a-ploskin/repos/TerraLabel/venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
idx2label = ['Barren', 'Forest', 'Agriculture', 'Road', 'Building', 'Water']

processor = MaskFormerImageProcessor(reduce_labels=True, ignore_index=255, do_resize=False, do_rescale=False, do_normalize=False)
model = MaskFormerForInstanceSegmentation.from_pretrained(
    "facebook/maskformer-swin-base-ade",
    id2label=idx2label,
    ignore_mismatched_sizes=True
)

Some weights of MaskFormerForInstanceSegmentation were not initialized from the model checkpoint at facebook/maskformer-swin-base-ade and are newly initialized because the shapes did not match:
- class_predictor.weight: found shape torch.Size([151, 256]) in the checkpoint and torch.Size([7, 256]) in the model instantiated
- class_predictor.bias: found shape torch.Size([151]) in the checkpoint and torch.Size([7]) in the model instantiated
- criterion.empty_weight: found shape torch.Size([151]) in the checkpoint and torch.Size([7]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [3]:
img_dir = '/home/a-ploskin/repos/TerraLabel/data/task_0/data'
masks_dir = '/home/a-ploskin/repos/TerraLabel/data/masks'

train_ds, eval_ds = SemanticSegmentationDataset.get_train_and_eval_datasets(
    processor, img_dir, masks_dir
)

In [4]:
train_dataloader = DataLoader(train_ds, batch_size=2, shuffle=True, drop_last=True)
eval_dataloader = DataLoader(eval_ds, batch_size=2, shuffle=True, drop_last=True)

In [5]:
log_path = './logs'
writer = SummaryWriter(log_path)
metric = load("mean_iou")

In [ ]:
from datetime import datetime


trainer = SegmenterModeltrainer(
    model=model,
    device='cuda:1',
    train_dataloader=train_dataloader,
    eval_dataloader=eval_dataloader,
    metric=metric,
    writer=writer
)

file_path = f'./models/maskformer_{datetime.now()}'

segmenter = trainer.train(file_path=file_path, n_epochs=50)

  0%|          | 0/200 [00:00<?, ?it/s]

Epoch: 0


  0%|          | 0/200 [00:00<?, ?it/s]


AttributeError: 'list' object has no attribute 'squeeze_'